<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment

In [15]:
!pip install -q findspark
!pip install -q pyspark

!echo "finished :)"

finished :)


## Setup environment variables

In [43]:
!wget -P -q https://github.com/ja8an/big-data/blob/main/in/forestfireinput.csv?raw=true -O forestfireinput.csv
!wget -P -q https://github.com/ja8an/big-data/blob/main/in/bible.txt?raw=true -O bible.txt
!echo "finished"

--2022-10-19 22:27:56--  https://github.com/ja8an/big-data/blob/main/in/forestfireinput.csv?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ja8an/big-data/raw/main/in/forestfireinput.csv [following]
--2022-10-19 22:27:56--  https://github.com/ja8an/big-data/raw/main/in/forestfireinput.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv [following]
--2022-10-19 22:27:56--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTT

## Setup Spark

In [3]:
import findspark
findspark.init()

In [28]:
from pyspark import SparkConf, SparkContext, SparkFiles
conf = SparkConf().setAppName("teste").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)
"spark setup"

'spark setup'

In [ ]:
!git clone https://github.com/ja8an/big-data.git

# Activities

## WordCount

In [44]:
textFile = sc.textFile("/content/bible.txt")
words = textFile.flatMap(lambda line : line.split(" "))
wordCounts = words.map(lambda word : (word, 1))
count = wordCounts.reduceByKey(lambda a, b: a + b)

count.take(5)

[('God', 2229), ('created', 38), ('', 27159), ('And', 12735), ('earth', 328)]

## Forest Fire Mean

In [45]:
dataset = sc.textFile("/content/forestfireinput.csv")

lines = dataset.map(lambda line : line.split(","))
lines = lines.map(lambda cols: float(cols[4]))

media = lines.reduce(lambda a, b: a + b)

average = media / lines.count()

average

90.64468085106387

In [47]:
dataset = sc.textFile("/content/airports.txt")

lines = dataset.map(lambda x : x.split("\t"))
filter = lines.filter(lambda x: x[3] == 'United States' and float(x[6]) > 40.0)
cols = filter.map(lambda x: ','.join(x).upper())
cols.take(1)

['6890,DOWAGIAC MUNICIPAL AIRPORT,DOWAGIAC,UNITED STATES,C91,\\N,41.9929342,-86.1280125,748,-5,U,AMERICA/NEW_YORK']

In [49]:
dataset = sc.textFile("/content/airports.txt")

lines = dataset.map(lambda x : x.split("\t"))
filter = lines.filter(lambda x: x[3] == 'United States')
cols = filter.map(lambda x: float(x[6]))
sum = cols.reduce(lambda x, y: x + y)
sum / filter.count()

41.44429158866408

In [62]:
dataset = sc.textFile("/content/RealState.csv")

lines = dataset.map(lambda x: x.split(',')).filter(lambda x: x[0] != "MLS")

lines.take(2)

[['132842',
  'Arroyo Grande',
  '795000.00',
  '3',
  '3',
  '2371',
  '335.30',
  'Short Sale'],
 ['134364',
  'Paso Robles',
  '399000.00',
  '4',
  '3',
  '2818',
  '141.59',
  'Short Sale']]